<a href="https://colab.research.google.com/github/JimMcDonough/production_repo/blob/main/Roberta_Full_with_adver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Dec  9 18:06:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0    50W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.9 MB/s 
     |████████████████████████████████| 182 kB 94.6 MB/s 
     |████████████████████████████████| 7.6 MB 76.6 MB/s 


In [5]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.5 MB/s 
     |████████████████████████████████| 212 kB 99.1 MB/s 
     |████████████████████████████████| 132 kB 95.3 MB/s 
     |████████████████████████████████| 127 kB 96.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [6]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import QAutils

In [7]:
#Set your paths here
SQuAD_Dataset_Path = r"/content/drive/My Drive/sample_data/train-v2.0.json"
saveDirectory = r"/content/drive/My Drive/sample_data"

In [11]:
_,_,_ = QAutils.createTrainValTestSplit(dataPath = SQuAD_Dataset_Path, saveDirectory=saveDirectory)

In [9]:
from datasets import load_dataset, Value, Sequence, Features

data_files={"train": saveDirectory + r"/train.json",
           "validation": saveDirectory + r"/val.json",
           "test":  saveDirectory + r"/test.json"}

In [12]:
features = Features({'answers': [{'text': Value(dtype='string'), 'answer_start': Value(dtype='int32')}],
'context': Value(dtype='string'),
'id': Value(dtype='string'),
'question': Value(dtype='string'),
'title': Value(dtype='string')})

In [13]:
dataset = load_dataset("json", data_files=data_files, features=features, cache_dir = saveDirectory )

Extracting data files #2:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /content/drive/My Drive/sample_data/json/default-d2266695f2747bb2/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [15]:
from transformers import DefaultDataCollator
from transformers import TFAutoModelForQuestionAnswering
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers.schedules import PolynomialDecay
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

data_collator = DefaultDataCollator(return_tensors="tf")

In [16]:
max_length = 384
stride = 128
batch_size = 16
num_epochs = 1
learning_rate = 0.000064


In [17]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",  
        stride = stride,   #only truncates the context nothing else
        return_overflowing_tokens=True,
        return_offsets_mapping=True,  # map the start and end positions of the answer to the original context 
        padding="max_length",
    )
    
    offset_mapping = inputs.pop("offset_mapping")
    #print(offset_mapping)
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []
    #print(answers)
    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        #start_char = answer["answer_start"][0]
        #end_char = answer["answer_start"][0] + len(answer["text"][0])  #start char + length giver chars with answer
        start_char = answer[0]["answer_start"]
        end_char = answer[0]["answer_start"] + len(answer[0]["text"])  
        sequence_ids = inputs.sequence_ids(i)  #no clue what this does
        #print(answer, end_char, sequence_ids)
        # Find the start and end of the context        ; not sure how this section works
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1
    
        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char: 
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)
            
            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)
            
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs 

In [18]:
tokenized_squad = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)


  0%|          | 0/106 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

In [19]:
model = TFAutoModelForQuestionAnswering.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/657M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForQuestionAnswering.

Some layers of TFRobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_squad["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_squad["validation"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

num_train_steps = len(tf_train_set) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=learning_rate, end_learning_rate=0.0, decay_steps=num_train_steps
)

opt = Adam(learning_rate=lr_scheduler)

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

In [21]:
model.fit(tf_train_set, validation_data=tf_validation_set, epochs=num_epochs)

6679/6679 [==============================] - 1115s 165ms/step - loss: 2.2182 - end_logits_loss: 1.0904 - start_logits_loss: 1.1278 - end_logits_accuracy: 0.6715 - start_logits_accuracy: 0.6553 - val_loss: 1.5732 - val_end_logits_loss: 0.7553 - val_start_logits_loss: 0.8180 - val_end_logits_accuracy: 0.7605 - val_start_logits_accuracy: 0.7377


In [22]:
#https://huggingface.co/docs/transformers/v4.23.1/en/serialization
model.save_pretrained(saveDirectory + r"/model/")
tokenizer.save_pretrained(saveDirectory + r"/model/")

('/content/drive/My Drive/sample_data/model/tokenizer_config.json',
 '/content/drive/My Drive/sample_data/model/special_tokens_map.json',
 '/content/drive/My Drive/sample_data/model/vocab.json',
 '/content/drive/My Drive/sample_data/model/merges.txt',
 '/content/drive/My Drive/sample_data/model/added_tokens.json',
 '/content/drive/My Drive/sample_data/model/tokenizer.json')